In [1]:
import psycopg2

# 连接到数据库
conn = psycopg2.connect(
    database="fomostop",
    user="postgres",
    password="l5F|XSfOdY]rYdZ?",
    host="192.168.123.88",
    port="5432"
)

# 创建一个数据库游标
cur = conn.cursor()

# 替换日期范围和日期格式
bdate = "2023_08_21"
edate = "2023_08_31"
date_format = "_\d{4}_\d{2}_\d{2}_"
types = 'stocks'

# 构建查询字符串
query = """
    SELECT DISTINCT ON (substring(table_name, 2, 10)) table_name
    FROM information_schema.tables
    WHERE table_name ~ %s
      AND table_name >= %s AND table_name <= %s
    ORDER BY substring(table_name, 2, 10) DESC, table_name DESC;
"""

# 执行查询
cur.execute(query, (f'{date_format}{types}', f'_{bdate}_', f'_{edate}_'))

# 获取查询结果
last_tables = cur.fetchall()




latest_daily_tables = [table[0] for table in last_tables]
latest_daily_tables

['_2023_08_30_stocks_1693424518',
 '_2023_08_29_stocks_1693338137',
 '_2023_08_28_stocks_1693252137',
 '_2023_08_25_stocks_1692992516',
 '_2023_08_24_stocks_1692906694',
 '_2023_08_23_stocks_1692819779',
 '_2023_08_22_stocks_1692733342',
 '_2023_08_21_stocks_1692631566']

In [2]:
import psycopg2
import pandas as pd

# 假设这些是你的条件变量
ticker = 'AAPL'
otypes = 'Call'
strike = 200
exp_date = '2023-11-17'

# 创建一个空的DataFrame
main_df = pd.DataFrame()

# 针对每个日期的表格进行操作
for table_name in latest_daily_tables:

    # 连接到数据库
    conn = psycopg2.connect(
        database="fomostop",
        user="postgres",
        password="l5F|XSfOdY]rYdZ?",
        host="192.168.123.88",
        port="5432"
    )

    # 创建一个数据库游标
    cur = conn.cursor()

    # 构建查询字符串，用于从表格中提取数据
    query = f"""
        SELECT * FROM {table_name}
        WHERE "Symbol" = %s AND "Type" = %s AND "Strike" = %s AND "Exp Date" = %s;
    """

    # 执行查询并将结果存储到一个DataFrame中
    cur.execute(query, (ticker, otypes, strike, exp_date))
    data = cur.fetchall()
    data = pd.DataFrame(data, columns=["Symbol", "Price", "Type", "Strike", "Exp Date", "DTE", "Bid", "Midpoint", "Ask", "Last", "Initiator", "Volume", "Open Int", "OI Chg", "IV", "Time"])

    # 在数据中添加一个名为 'Date' 的新列，并将日期赋给它
    data['Date'] = pd.to_datetime(table_name[1:11], format='%Y_%m_%d')

    # 将 'IV' 列中的 Decimal 数据转换为浮点数
    data['IV'] = data['IV'].apply(float)

    # 使用round()函数将 'IV' 列保留到小数点后 4 位
    data['IV'] = data['IV'].round(4)

    # 使用concat函数将数据合并到主DataFrame中
    main_df = pd.concat([main_df, data])

    # 关闭游标和数据库连接
    cur.close()
    conn.close()

# 现在，main_df 包含了所有日期中满足条件的数据，'Date' 列已经转换为日期格式，'IV' 列保留到小数点后 4 位
main_df.set_index('Date', inplace=True)
main_df

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Initiator,Volume,Open Int,OI Chg,IV,Time
Date,,,,,,,,,,,,,,,,
2023-08-30,AAPL,187.63,Call,200.0,2023-11-17,79.0,3.35,3.38,3.4,3.38,Neutral,7380.0,38522.0,6567.0,0.2080,15:13 ET
2023-08-29,AAPL,184.68,Call,200.0,2023-11-17,80.0,2.62,2.63,2.63,2.62,Seller,8734.0,31955.0,1410.0,0.2071,15:20 ET
2023-08-28,AAPL,180.49,Call,200.0,2023-11-17,81.0,1.66,1.67,1.68,1.67,Neutral,2192.0,30545.0,2245.0,0.2099,15:16 ET
2023-08-25,AAPL,178.95,Call,200.0,2023-11-17,84.0,1.54,1.55,1.55,1.53,Aggressive Seller,2920.0,28300.0,-1774.0,0.2091,15:14 ET
2023-08-24,AAPL,176.27,Call,200.0,2023-11-17,85.0,1.36,1.37,1.37,1.37,Neutral,1426.0,30074.0,2662.0,0.2136,15:27 ET
2023-08-23,AAPL,181.47,Call,200.0,2023-11-17,86.0,1.94,1.94,1.95,1.95,Buyer,5324.0,27412.0,224.0,0.2031,15:13 ET
2023-08-22,AAPL,177.11,Call,200.0,2023-11-17,87.0,1.33,1.34,1.35,1.32,Aggressive Seller,666.0,27188.0,118.0,0.2035,15:10 ET


In [6]:
import psycopg2
import pandas as pd

# 假设这些是你的条件变量
ticker = 'AAPL'
otypes = 'Call'
strike = 200
exp_date = '2023-11-17'
bdate = "2023_08_21"
edate = "2023_08_31"
date_format = "_\d{4}_\d{2}_\d{2}_"
types = 'stocks'

# 创建一个空的DataFrame
main_df = pd.DataFrame()

# 连接到数据库
conn = psycopg2.connect(
    database="fomostop",
    user="postgres",
    password="l5F|XSfOdY]rYdZ?",
    host="192.168.123.88",
    port="5432"
)

# 创建一个数据库游标
cur = conn.cursor()

# 构建查询字符串，用于从表格中提取数据
query_1 = """
    SELECT DISTINCT ON (substring(table_name, 2, 10)) table_name
    FROM information_schema.tables
    WHERE table_name ~ %s
      AND table_name >= %s AND table_name <= %s
    ORDER BY substring(table_name, 2, 10) DESC, table_name DESC;
"""

# 执行查询
cur.execute(query_1, (f'{date_format}{types}', f'_{bdate}_', f'_{edate}_'))

# 获取查询结果
last_tables = cur.fetchall()

latest_daily_tables = [table[0] for table in last_tables]

# 针对每个日期的表格进行操作
for table_name in latest_daily_tables:
    # 构建查询字符串，用于从表格中提取数据
    query_2 = f"""
        SELECT * FROM {table_name}
        WHERE "Symbol" = %s AND "Type" = %s AND "Strike" = %s AND "Exp Date" = %s;
    """

    # 执行查询并将结果存储到一个DataFrame中
    cur.execute(query_2, (ticker, otypes, strike, exp_date))
    data = cur.fetchall()
    data = pd.DataFrame(data, columns=["Symbol", "Price", "Type", "Strike", "Exp Date", "DTE", "Bid", "Midpoint", "Ask", "Last", "Initiator", "Volume", "Open Int", "OI Chg", "IV", "Time"])

    # 在数据中添加一个名为 'Date' 的新列，并将日期赋给它
    data['Date'] = pd.to_datetime(table_name[1:11], format='%Y_%m_%d')

    # 将 'IV' 列中的 Decimal 数据转换为浮点数
    data['IV'] = data['IV'].apply(float)

    # 使用round()函数将 'IV' 列保留到小数点后 4 位
    data['IV'] = data['IV'].round(4)

    # 使用concat函数将数据合并到主DataFrame中
    main_df = pd.concat([main_df, data])

# 关闭游标和数据库连接
cur.close()
conn.close()

# 现在，main_df 包含了所有日期中满足条件的数据，'Date' 列已经转换为日期格式，'IV' 列保留到小数点后 4 位
main_df.set_index('Date', inplace=True)
main_df


,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Initiator,Volume,Open Int,OI Chg,IV,Time
Date,,,,,,,,,,,,,,,,
2023-08-30,AAPL,187.63,Call,200.0,2023-11-17,79.0,3.35,3.38,3.4,3.38,Neutral,7380.0,38522.0,6567.0,0.2080,15:13 ET
2023-08-29,AAPL,184.68,Call,200.0,2023-11-17,80.0,2.62,2.63,2.63,2.62,Seller,8734.0,31955.0,1410.0,0.2071,15:20 ET
2023-08-28,AAPL,180.49,Call,200.0,2023-11-17,81.0,1.66,1.67,1.68,1.67,Neutral,2192.0,30545.0,2245.0,0.2099,15:16 ET
2023-08-25,AAPL,178.95,Call,200.0,2023-11-17,84.0,1.54,1.55,1.55,1.53,Aggressive Seller,2920.0,28300.0,-1774.0,0.2091,15:14 ET
2023-08-24,AAPL,176.27,Call,200.0,2023-11-17,85.0,1.36,1.37,1.37,1.37,Neutral,1426.0,30074.0,2662.0,0.2136,15:27 ET
2023-08-23,AAPL,181.47,Call,200.0,2023-11-17,86.0,1.94,1.94,1.95,1.95,Buyer,5324.0,27412.0,224.0,0.2031,15:13 ET
2023-08-22,AAPL,177.11,Call,200.0,2023-11-17,87.0,1.33,1.34,1.35,1.32,Aggressive Seller,666.0,27188.0,118.0,0.2035,15:10 ET
